### Import libraries

In [1]:
# for data
import pandas as pd  
import numpy as np  

# for plotting
import matplotlib.pyplot as plt 
import seaborn as sns  
import folium  
from folium import plugins
import plotly.express as px  

# for graph and routing
import osmnx as ox  
import networkx as nx 
import geopandas as gpd
# for advanced routing
# from ortools.constraint_solver import pywrapcp  
# from ortools.constraint_solver import routing_enums_pb2

import re
import random


### import chicago data

In [2]:
df = pd.read_csv('chicago_data_5.2_perc_sample.csv')

In [685]:
df.shape

(4528691, 32)

In [686]:
df.head(3)

,Unnamed: 0,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,percent_time_chicago,percent_distance_chicago,pickup_census_tract,dropoff_census_tract,pickup_community_area,...,rhum,prcp,wdir,wpgt,pres,year,month,dayname,hour_start,hour_end
0,453863,2023-05-14 09:00:00,2023-05-14 09:00:00,314.0,1.37,0.9937,0.9920,1.703183e+10,1.703106e+10,7.0,...,65.0,0.0,20.0,NaN,1024.0,2023,5,Sunday,9,9
1,681723,2023-05-06 22:15:00,2023-05-06 22:15:00,615.0,2.43,0.9985,1.0000,1.703132e+10,1.703183e+10,32.0,...,47.0,0.0,150.0,NaN,1012.5,2023,5,Saturday,22,22
2,548264,2023-05-09 10:15:00,2023-05-09 10:45:00,1475.0,13.36,0.9985,0.9956,1.703105e+10,1.703198e+10,5.0,...,89.0,0.0,340.0,NaN,1018.0,2023,5,Tuesday,10,10


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4528691 entries, 0 to 4528690
Data columns (total 27 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Unnamed: 0                 int64  
 1   trip_start_timestamp       object 
 2   trip_end_timestamp         object 
 3   trip_seconds               float64
 4   trip_miles                 float64
 5   percent_time_chicago       float64
 6   percent_distance_chicago   float64
 7   pickup_census_tract        float64
 8   dropoff_census_tract       float64
 9   pickup_community_area      float64
 10  dropoff_community_area     float64
 11  fare                       float64
 12  tip                        float64
 13  additional_charges         float64
 14  trip_total                 float64
 15  shared_trip_authorized     bool   
 16  shared_trip_match          bool   
 17  trips_pooled               int64  
 18  pickup_centroid_location   object 
 19  dropoff_centroid_location  object 
 20  te

In [4]:
# drop na values in pickup_centroid_location or dropoff_centroid_location
data=df.dropna(subset=['pickup_centroid_location', 'dropoff_centroid_location'])
len(data)

3741872

In [5]:
data[['pickup_centroid_location', 'dropoff_centroid_location']].isnull().sum()

pickup_centroid_location     0
dropoff_centroid_location    0
dtype: int64

In [9]:
data.head()

,Unnamed: 0,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,percent_time_chicago,percent_distance_chicago,pickup_census_tract,dropoff_census_tract,pickup_community_area,...,trips_pooled,pickup_centroid_location,dropoff_centroid_location,temp,dwpt,rhum,prcp,wdir,wpgt,pres
0,453863,2023-05-14T09:00:00.000,2023-05-14T09:00:00.000,314.0,1.37,0.9937,0.9920,1.703183e+10,1.703106e+10,7.0,...,1,POINT (-87.6640782395 41.9218774613),POINT (-87.6467297286 41.9346591566),14.4,7.9,65.0,0.0,20.0,NaN,1024.0
1,681723,2023-05-06T22:15:00.000,2023-05-06T22:15:00.000,615.0,2.43,0.9985,1.0000,1.703132e+10,1.703183e+10,32.0,...,1,POINT (-87.6219716519 41.8774061234),POINT (-87.6540926517 41.8716894745),25.0,12.9,47.0,0.0,150.0,NaN,1012.5
2,548264,2023-05-09T10:15:00.000,2023-05-09T10:45:00.000,1475.0,13.36,0.9985,0.9956,1.703105e+10,1.703198e+10,5.0,...,1,POINT (-87.6835062305 41.9504425991),POINT (-87.9030396611 41.9790708201),8.9,7.2,89.0,0.0,340.0,NaN,1018.0
3,199697,2023-05-12T17:30:00.000,2023-05-12T18:00:00.000,1413.0,6.52,0.9995,0.9880,1.703106e+10,1.703108e+10,6.0,...,1,POINT (-87.6665362813 41.9506733576),POINT (-87.6318639497 41.8920421365),25.0,16.7,60.0,0.0,140.0,NaN,1016.0
4,165024,2023-05-08T20:30:00.000,2023-05-08T20:45:00.000,335.0,1.33,1.0000,1.0000,1.703184e+10,1.703141e+10,41.0,...,1,POINT (-87.6012851221 41.7904693995),POINT (-87.5853031602 41.8012268363),12.8,11.0,89.0,2.1,30.0,NaN,1014.0


In [6]:
data.groupby(['pickup_centroid_location', 'dropoff_centroid_location'])[['trip_miles']].count()

trip_miles
pickup_centroid_location             dropoff_centroid_location                       
POINT (-87.529950466 41.6954498278)  POINT (-87.548564529 41.7319988447)            1
                                     POINT (-87.600092686 41.7008887972)            1
                                     POINT (-87.6318639497 41.8920421365)           1
POINT (-87.5307124836 41.7030053028) POINT (-87.529950466 41.6954498278)            1
                                     POINT (-87.5470098641 41.7413866268)           1
...                                                                               ...
POINT (-87.913624596 41.9802643146)  POINT (-87.7980321814 41.9292973683)         106
                                     POINT (-87.8045320063 41.9850151008)         910
                                     POINT (-87.8060200025 41.9465114202)         597
                                     POINT (-87.8137810343 42.0076125931)         198
                                     POINT (-87.913624596 41.9802643146)         2569

[189373 rows x 1 columns]

### Match the actual trip route with the shortest route calculated from networkx

In [3]:
# create a osmnx graph for Chicago city

place = 'Chicago, Illinois, USA'
G = ox.graph_from_place(place, network_type='drive')

#add speed and travel_time attributes to the edges.
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)

In [4]:
# create geo-dataframe for both nodes and edges
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)

In [ ]:
# create a dataframe data_group to only include the unique 
# ('pickup_centroid_location', 'dropoff_centroid_location') from the dataframe data
# to avoid duplicated calculations in the below
data_group=data.drop_duplicates(['pickup_centroid_location', 'dropoff_centroid_location'])

In [202]:
#convert the end points of each pair into nodes of the graph:
data_group['start_node']=data_group.apply(lambda row: 
                                                        ox.distance.nearest_nodes(
                                                            G, 
                                                            float(re.findall(pattern, row['pickup_centroid_location'])[0]),
                                                            float(re.findall(pattern, row['pickup_centroid_location'])[1])),
                                                            axis=1)
data_group['end_node']=data_group.apply(lambda row: 
                                                        ox.distance.nearest_nodes(
                                                            G, 
                                                            float(re.findall(pattern, row['dropoff_centroid_location'])[0]),
                                                            float(re.findall(pattern, row['dropoff_centroid_location'])[1])),
                                                            axis=1)

In [ ]:
# Calculate the distance between the point and the nearest node
data_group['start_node_distance'] = data_group.apply(lambda row: 
                                                                       ox.distance.great_circle_vec(
                                                                       float(re.findall(pattern, row['pickup_centroid_location'])[1]),
                                                                       float(re.findall(pattern, row['pickup_centroid_location'])[0]),
                                                                       G.nodes[row['start_node']]['y'], G.nodes[row['start_node']]['x']),
                                                                   axis=1)
data_group['end_node_distance'] = data_group.apply(lambda row: 
                                                                       ox.distance.great_circle_vec(
                                                                       float(re.findall(pattern, row['dropoff_centroid_location'])[1]),
                                                                       float(re.findall(pattern, row['dropoff_centroid_location'])[0]),
                                                                       G.nodes[row['end_node']]['y'], G.nodes[row['end_node']]['x']),
                                                                   axis=1)

In [ ]:
# calculate the shortest route by length
def get_shortest_route(row):
    try:
        shortest_route=nx.shortest_path(G, row['start_node'], row['end_node'], weight='length')
        shortest_route_length=nx.shortest_path_length(G, row['start_node'], row['end_node'], weight='length')   
    except nx.NetworkXNoPath:
        shortest_route=np.nan
        shortest_route_length=np.nan
    return shortest_route, shortest_route_length
# Calculate the shortest route along with its length
data_group[['shortest_route','shortest_route_length']]= data_group.apply(
                                                                                                                                 lambda row:
                                                                                                                                pd.Series(get_shortest_route(row)), axis=1)

In [432]:
# save this dataframe for an easier access in the future
data_group.to_csv('Chicago_data_group.csv')

In [433]:
# merge the 'start_node' and 'end_node' columns with the original data dataframe and renamed it as data_merged
data_group_col=data_group[['pickup_centroid_location', 'dropoff_centroid_location', 'start_node','end_node',
                           'start_node_distance', 'end_node_distance','shortest_route', 'shortest_route_length']]
data_merged=pd.merge(data, data_group_col, on=['pickup_centroid_location', 'dropoff_centroid_location'])

In [ ]:
# add a new column about the absolute difference between the actual trip_miles and the calculated shortest route length
data_merged['absdiff_trip_shortest_meters']=abs(data_merged['trip_miles']*1609.344-data_merged['shortest_route_length'])

In [434]:
# save this dataframe for an easier access in the future
data_merged.to_csv('Chicago_orig.csv')